In [1]:
#THE CLIENT PART IS GOING TO RECEIVE ONLY A FRAGMENT OF THE IMAGE, 
#THE FRAGMENT CONTAINING THE ACTUAL PLAY AREA
#BECAUSE OF THIS WE'LL ONLY NEED:
#GET HEALTH (and it's necesary ops)
#GET PLAYERS POS (and it's necesary ops)

In [1]:
import random
import datetime, time
import socket, sys, cv2, numpy, time
from PIL import Image
import tensorflow as tf
import tensorflow.contrib.slim as slim
import os
import csv
import datetime, time
import numpy as np

In [2]:
def isGreen(rgb):
    r, g, b = rgb
    if (g>r) and (g>b):
        return True
    return False

def isBlue(rgb):
    r, g, b = rgb
    if (b>r) and (b>g):
        return True
    return False

def isRed(rgb):
    r, g, b = rgb
    if (r>b) and (r>g):
        return True
    return False

In [4]:
def getHealths(im, coord_p1, coord_p2, right_end):
    p1 = 0
    p2 = 0
    #verde es #3CD400 == rgb(60,212,0) aproximadamente, isGreen comprueba que G sea el valor principal
    for x in range(right_end+1):
        if (isGreen(im.getpixel(coord_p1))):
            p1 = p1 + 1
        if (isGreen(im.getpixel(coord_p2))):
            p2 = p2 + 1
        coord_p1 = x, y = coord_p1[0]+1, coord_p1[1]
        coord_p2 = x, y = coord_p2[0]+1, coord_p2[1]
    return(p1,p2)

def is_game_over(HP_1, HP_2, end_time):
    if (HP_1 == 0):
        return 1 #P1 LOOSES
    elif (HP_2 == 0):
        return -1 #P2 LOOSES
    elif ((HP_1 == 0) and (HP_2 == 0)) or (datetime.datetime.now() > end_time):
        return 2 #DRAW
    return 0

def is_game_over_neutral_endless(HP_1, HP_2):
    if (HP_1 == 0):
        return 1 #P1 LOOSES
    elif (HP_2 == 0):
        return 2 #P2 LOOSES
    return 0

In [5]:
def translate_settings(buffer):
    p1_x_str = ''
    p2_x_str = ''
    p_y_str = ''
    length_str = ''
    control = 0
    for i in buffer:
        if (control == 0):
            if (i==';'):
                control +=1
            else:
                p1_x_str += i
        elif (control == 1):
            if (i==';'):
                control +=1
            else:
                p2_x_str += i
        elif (control == 2):
            if (i==';'):
                control +=1
            else:
                p_y_str += i
        elif (control == 3):
            length_str += i
    return(int(p1_x_str), int(p2_x_str), int(p_y_str), int(length_str))

In [6]:
#REGLAS: NADA DE PONERSE A ENTRENAR EN LA NOCHE ANTES DE QUE SE ACABE EL MES!!!
#SI LO HACES TE DOY UNA PALIZA
#Y SI YA TE PONES A HACERLO EN AÑO NUEVO TE MATO
def setTime():
    dt = datetime.datetime.now()
    if (dt.second % 10 > 5):
        if (dt.second>54): #Hay que pasar al siguiente minuto
            if (dt.minute==59): #Hay que pasar a la siguiente hora
                if (dt.hour == 23): #Hay que pasar al siguiente día
                    dia = dt.day+1
                    hora = 0
                    minuto = 0
                    sec = 5
                    dt.replace(day=dia, hour=hora, minute=minuto, second=sec)
                else: #hora nueva
                    hora = dt.hour+1
                    minuto = 0
                    sec = 5
                    dt.replace(hour=hora, minute=minuto, second=sec)
            else: #minuto nuevo
                minuto = dt.minute+1
                sec = 5
                dt = dt.replace(minute=minuto ,second=sec)
        else: #caso "simple"
            sec = (((dt.second // 10)+1)*10) + 5 #Z+1 5
            dt = dt.replace(second=sec)
    else:
        if (dt.second>49): #Hay que pasar al siguiente minuto
            if (dt.minute==59): #Hay que pasar a la siguiente hora
                if (dt.hour == 23): #Hay que pasar al siguiente día
                    dia = dt.day+1
                    hora = 0
                    minuto = 0
                    sec = 5
                    dt.replace(day=dia, hour=hora, minute=minuto, second=sec)
                else: #hora nueva
                    hora = dt.hour+1
                    minuto = 0
                    sec = 0
                    dt.replace(hour=hora, minute=minuto, second=sec)
            else: #minuto nuevo
                minuto = dt.minute+1
                sec = 0
                dt = dt.replace(minute=minuto ,second=sec)
        else: #caso "simple"
            sec = ((dt.second // 10)+1)*10 #Z+1 0
            dt = dt.replace(second=sec)
    return dt

In [7]:
def addMicrosecs(microsecs):
    fulldate = datetime.datetime.now()
    fulldate = fulldate + datetime.timedelta(microseconds=microsecs)
    return fulldate

In [8]:
def addMinute():
    fulldate = datetime.datetime.now()
    fulldate = fulldate + datetime.timedelta(minutes=1)
    return fulldate

In [9]:
def recvall(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf

In [10]:
#The connection aspect of things

In [11]:
def connection_to_Monty(mensaje):
    PORT = 50006
    dir = "localhost"
    if (mensaje == "get"): #GET image
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        w = int(recvall(s,8))
        h = int(recvall(s,8))
        length = recvall(s,16)
        stringData = recvall(s, int(length))
        data = numpy.fromstring(stringData, dtype='uint8').reshape(h,w,3)
        im = Image.fromarray(data)
        s.close()
        return im
    elif (mensaje == "set"): #We set the initial values of padding needed, health pos
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        buf = s.recv(1024) #We will just need the p1_x, p2_x and y of setPadding()
        #####(x_pad, y_pad, p1_x, p2_x, y, length) = setPadding()#####
        print(buf.decode())
        s.close()
        (p1_x, p2_x, p_y, length) = translate_settings(buf.decode())
        coordinate_p1 = x, y = p1_x, p_y
        coordinate_p2 = x, y = p2_x, p_y
        return (coordinate_p1, coordinate_p2, length)
    elif (mensaje == "start2"): #START env and get wait time
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,(PORT-1)))
        s.sendall(mensaje.encode())
        buf = s.recv(1024)
        s.close()
        converted = buf.decode()
        if (converted == 'Yes                 '):
            date = setTime()
            now = datetime.datetime.now()
            while(now<=date):
                time.sleep(1)
                now = datetime.datetime.now()
        return converted
    elif (mensaje[0] == "2" ): #SEND COMMAND
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((dir,PORT))
        s.sendall(mensaje.encode())
        s.close()

In [12]:
def starter_sync():
    txt = connection_to_Monty("start2")
    while(txt == 'No                  '):
        txt = connection_to_Monty("start2")
        time.sleep(1)

In [13]:
actions = ["0", "1", "2", "3", "4", "5", "6", "7", "8", 
           "01", "02", "03", "04", "05", "06", "07", "08", 
           "10", "12", "13", "14", "15", "16", "17", "18",
           "20", "21", "23", "24", "25", "26", "27", "28",
           "30", "31", "32", "34", "35", "36", "37", "38",
           "40", "41", "42", "43", "45", "46", "47", "48",
           "50", "51", "52", "53", "54", "56", "57", "58",
           "60", "61", "62", "63", "64", "65", "67", "68", 
           "70", "71", "72", "73", "74", "75", "76", "78",
           "80", "81", "82", "83", "84", "85", "86", "87"]
num_actions = len(actions)

In [14]:
##Algunos parámetros generales
im_height = 394 #Full size
im_width = 599 #Full size
im_size_sides = im_height * im_width #Full size
im_size = im_size_sides * 3 #la anchura y altura de la imagen junto con los canales rgb #Full size

im_height_half = 197
im_width_half = 300
im_size_sides_half = im_height_half * im_width_half
im_size_half = im_size_sides_half * 3

## Creamos la red del agente

In [15]:
class Qnetwork():
    def __init__(self,h_size,rnn_cell,myScope, size=1):
        #The network recieves a frame from the game, flattened into an array.
        #It then resizes it and processes it through four convolutional layers.
        if (size == 1):
            self.scalarInput =  tf.placeholder(shape=[None,None,im_size],dtype=tf.float32) #FULL SIZE
            self.imageIn = tf.reshape(self.scalarInput,shape=[-1,im_height,im_width,3]) #FULL SIZE
            self.conv1 = slim.convolution2d( \
                inputs=self.imageIn,num_outputs=32,\
                kernel_size=[18,18],stride=[8,8],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv1')
            self.conv2 = slim.convolution2d( \
                inputs=self.conv1,num_outputs=64,\
                kernel_size=[10,10],stride=[4,4],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv2')
            self.conv3 = slim.convolution2d( \
                inputs=self.conv2,num_outputs=64,\
                kernel_size=[6,6],stride=[2,2],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv3')
            self.conv4 = slim.convolution2d( \
                inputs=self.conv3,num_outputs=h_size,\
                kernel_size=[3,3],stride=[6,6],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv4')
        
            self.trainLength = tf.placeholder(dtype=tf.int32)
            #We take the output from the final convolutional layer and send it to a recurrent layer.
            #The input must be reshaped into [batch x trace x units] for rnn processing, 
            #and then returned to [batch x units] when sent through the upper levels.
            self.batch_size = tf.placeholder(dtype=tf.int32,shape=[])
            self.convFlat = tf.reshape(self.conv4, [self.batch_size,self.trainLength,h_size])
            #For the previous RNN_CELL
            self.state_in = rnn_cell.zero_state(self.batch_size, tf.float32)
            #We create the RNN and it's initial state
            self.rnn,self.rnn_state = tf.nn.dynamic_rnn(\
                cell=rnn_cell,inputs=self.convFlat,initial_state=self.state_in,dtype=tf.float32,scope=myScope+'_rnn')
            self.rnn = tf.reshape(self.rnn,shape=[-1,h_size])
            #The output from the recurrent player is then split into separate Value and Advantage streams
            self.streamA,self.streamV = tf.split(self.rnn,2,1) #stream of Advantage, stream of Value
            self.AW = tf.Variable(tf.random_normal([h_size//2,len(actions)])) #Advantage weight/worth
            self.VW = tf.Variable(tf.random_normal([h_size//2,1])) #Value weight/worth
            self.Advantage = tf.matmul(self.streamA,self.AW)
            self.Value = tf.matmul(self.streamV,self.VW)
        
            self.salience = tf.gradients(self.Advantage,self.imageIn)
            #Then combine them together to get our final Q-values.
            self.Qout = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True))
            self.predict = tf.argmax(self.Qout,1)
        
            #Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
            self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
            self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
            self.actions_onehot = tf.one_hot(self.actions,num_actions,dtype=tf.float32)
            
            self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)
        
            self.td_error = tf.square(self.targetQ - self.Q)
        
            #In order to only propogate accurate gradients through the network, we will mask the first
            #half of the losses for each trace as per Lample & Chatlot 2016
            self.maskA = tf.zeros([self.batch_size,self.trainLength//2])
            self.maskB = tf.ones([self.batch_size,self.trainLength//2])
            self.mask = tf.concat([self.maskA,self.maskB],1)
            self.mask = tf.reshape(self.mask,[-1])
            self.loss = tf.reduce_mean(self.td_error * self.mask)
            
            self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
            self.updateModel = self.trainer.minimize(self.loss)
        #END IF  
        
        elif (size == 0.5):    
            self.scalarInput =  tf.placeholder(shape=[None,None,im_size_half],dtype=tf.float32) #FULL SIZE
            self.imageIn = tf.reshape(self.scalarInput,shape=[-1,im_height_half,im_width_half,3]) #FULL SIZE
            self.conv1 = slim.convolution2d( \
                inputs=self.imageIn,num_outputs=32,\
                kernel_size=[14,14],stride=[6,6],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv1')
            self.conv2 = slim.convolution2d( \
                inputs=self.conv1,num_outputs=64,\
                kernel_size=[10,10],stride=[3,3],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv2')
            self.conv3 = slim.convolution2d( \
                inputs=self.conv2,num_outputs=64,\
                kernel_size=[6,6],stride=[1,1],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv3')
            self.conv4 = slim.convolution2d( \
                inputs=self.conv3,num_outputs=h_size,\
                kernel_size=[3,3],stride=[6,6],padding='VALID', \
                biases_initializer=None,scope=myScope+'_conv4')
        
            self.trainLength = tf.placeholder(dtype=tf.int32)
            #We take the output from the final convolutional layer and send it to a recurrent layer.
            #The input must be reshaped into [batch x trace x units] for rnn processing, 
            #and then returned to [batch x units] when sent through the upper levels.
            self.batch_size = tf.placeholder(dtype=tf.int32,shape=[])
            self.convFlat = tf.reshape(self.conv4, [self.batch_size,self.trainLength,h_size])
            #For the previous RNN_CELL
            self.state_in = rnn_cell.zero_state(self.batch_size, tf.float32)
            #We create the RNN and it's initial state
            self.rnn,self.rnn_state = tf.nn.dynamic_rnn(\
                cell=rnn_cell,inputs=self.convFlat,initial_state=self.state_in,dtype=tf.float32,scope=myScope+'_rnn')
            self.rnn = tf.reshape(self.rnn,shape=[-1,h_size])
            #The output from the recurrent player is then split into separate Value and Advantage streams
            self.streamA,self.streamV = tf.split(self.rnn,2,1) #stream of Advantage, stream of Value
            self.AW = tf.Variable(tf.random_normal([h_size//2,len(actions)])) #Advantage weight/worth
            self.VW = tf.Variable(tf.random_normal([h_size//2,1])) #Value weight/worth
            self.Advantage = tf.matmul(self.streamA,self.AW)
            self.Value = tf.matmul(self.streamV,self.VW)
        
            self.salience = tf.gradients(self.Advantage,self.imageIn)
            #Then combine them together to get our final Q-values.
            self.Qout = self.Value + tf.subtract(self.Advantage,tf.reduce_mean(self.Advantage,axis=1,keep_dims=True))
            self.predict = tf.argmax(self.Qout,1)
        
            #Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
            self.targetQ = tf.placeholder(shape=[None],dtype=tf.float32)
            self.actions = tf.placeholder(shape=[None],dtype=tf.int32)
            self.actions_onehot = tf.one_hot(self.actions,num_actions,dtype=tf.float32)
            
            self.Q = tf.reduce_sum(tf.multiply(self.Qout, self.actions_onehot), axis=1)
        
            self.td_error = tf.square(self.targetQ - self.Q)
        
            #In order to only propogate accurate gradients through the network, we will mask the first
            #half of the losses for each trace as per Lample & Chatlot 2016
            self.maskA = tf.zeros([self.batch_size,self.trainLength//2])
            self.maskB = tf.ones([self.batch_size,self.trainLength//2])
            self.mask = tf.concat([self.maskA,self.maskB],1)
            self.mask = tf.reshape(self.mask,[-1])
            self.loss = tf.reduce_mean(self.td_error * self.mask)
            
            self.trainer = tf.train.AdamOptimizer(learning_rate=0.0001)
            self.updateModel = self.trainer.minimize(self.loss)
        #END IF

Experience replay

In [16]:
class experience_buffer():
    def __init__(self, buffer_size = 100):
        self.buffer = []
        self.buffer_size = buffer_size
    
    def add(self,experience):
        if len(self.buffer) + 1 >= self.buffer_size:
            self.buffer[0:(1+len(self.buffer))-self.buffer_size] = []
        self.buffer.append(experience)
            
    def sample(self,batch_size,trace_length):
        sampled_episodes = random.sample(self.buffer,batch_size)
        sampledTraces = []
        for episode in sampled_episodes:
            point = np.random.randint(0,len(episode)+1-trace_length)
            sampledTraces.append(episode[point:point+trace_length])
        sampledTraces = np.array(sampledTraces)
        return np.reshape(sampledTraces,[batch_size*trace_length,5])

In [17]:
#A bit of added help
def updateTargetGraph(tfVars, tau):
    total_vars = len(tfVars)
    op_holder = []
    for idx,var in enumerate(tfVars[0:total_vars//2]):
        op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau) + ((1-tau)*tfVars[idx+total_vars//2].value())))
    return op_holder

def updateTarget(op_holder, sess):
    for op in op_holder:
        sess.run(op)
    total_vars = len(tf.trainable_variables())
    a = tf.trainable_variables()[0].eval(session=sess)
    b = tf.trainable_variables()[total_vars//2].eval(session=sess)

In [18]:
def count_movements(actionBuf):
    total = 0
    divisor = 0
    for x in actionBuf:
        divisor += 1
        check = True
        if (int(x[0])<4) and (check):
                check = False
                total +=1
    return total/divisor

def count_blocks(actionBuf):
    total = 0
    divisor = 0
    for x in actionBuf:
        divisor += 1
        if x[0] == '4':
            total += 1
    return total/divisor

In [19]:
def calculate_reward_p2(status, actionBuff, current_length, hp1, hp2, flavour="vanilla"):
    if (flavour == "vanilla"):
        #Simple reward: IF WIN = 1 ; IF LOOSE = -1 ; IF DRAW = (hp2-hp1)/200 ; 200 == total health
        #The other additions are in case we want to make a more complex 
        # FOR PLAYER TWO: 2 = draw ; 1 = WIN ; -1 = LOOSE (coincidentally the rewards coincide with the status, minus the draw)
        if status == 2:
            return (hp2-hp1)/200
        elif status == 0:
            return((hp2-hp1)/200)/100 #ENCOURAGEMENT
        else:
            return status
    elif (flavour == "aggro"):
        if status == 2:
            return (hp2-hp1)/200
        elif status == 1:
            return (status * (68 / current_length))
        elif status == 0:
            return((hp2-hp1)/200)/100 #ENCOURAGEMENT
        else:
            return status
    elif (flavour == "kite"):
        movements = count_movements(actionBuff)
        if status == 2:
            return ((hp2-hp1)/200)*movements
        elif status == 1:
            return status*movements
        elif status == 0:
            return((hp2-hp1)/200)/100 #ENCOURAGEMENT
        else:
            return status
    elif (flavour == "tank"):
        blocks = count_blocks(actionBuff)
        if status == 2:
            return ((hp2-hp1)/200)*blocks
        elif status == 1:
            return status*blocks
        elif status == 0:
            return((hp2-hp1)/200)/100 #ENCOURAGEMENT
        else:
            return status

Establecer datos necesarios

In [20]:
actions_per_second = 2.5

In [25]:
#Setting the training parameters
batch_size = 4 #How many experience traces to use for each training step.
trace_length = 8 #How long each experience trace will be when training
update_freq = 5 #How often to perform a training step.
y = .99 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action
max_epLength = 60 * actions_per_second #The max allowed length of our episode.
anneling_steps = 200 * max_epLength #How many steps of training to reduce startE to endE.
num_episodes = 700 #How many episodes of game environment to train network with.
pre_train_steps = 100 * max_epLength #How many steps of random actions before training begins.
load_model = False #Whether to load a saved model.
path = "./drqn_2_02" #The path to save our model to.
#path = "./drqn_2_02_AGGRO"
#path = "./drqn_2_02_KITE"

flavour = "vanilla"
#flavour = "aggro"
#flavour = "kite"

h_size = 512 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
summaryLength = 100 #Number of epidoes to periodically save for analysis
tau = 0.001

In [26]:
(coordinate_p1, coordinate_p2, length) = connection_to_Monty("set")

17;376;25;200       


## Entrenar la red

Fase de entrenamiento del modelo de la red

In [27]:
tf.reset_default_graph()
#We define the cells for the primary and target q-networks
cell = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
cellT = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
mainQN = Qnetwork(h_size,cell,'main', size=0.5)
targetQN = Qnetwork(h_size,cellT,'target', size=0.5)

init = tf.global_variables_initializer()

saver = tf.train.Saver(max_to_keep=5)

trainables = tf.trainable_variables()

targetOps = updateTargetGraph(trainables,tau)

myBuffer = experience_buffer()

#Set the rate of random action decrease. 
e = startE
stepDrop = (startE - endE)/anneling_steps

#create lists to contain total rewards and steps per episode
jList = []
rList = []
total_steps = 0

#Make a path for our model to be saved in.
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
print(datetime.datetime.now())

In [ ]:
time.sleep(5)
with tf.Session() as sess:
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(path)
        if (ckpt == None):
            print('There is no model to load!')
        else:
            saver.restore(sess,ckpt.model_checkpoint_path)
    sess.run(init)
   
    updateTarget(targetOps,sess) #Set the target network to be equal to thagfslodfpkyfpesa primary network.
    for i in range(num_episodes):
        episodeBuffer = []
        #Reset environment and get first new observation
        actionBuffer = [] #Buffer of actions
        starter_sync() #El proceso terminará a tiempo para que sea un inicio sincronizado
        s = connection_to_Monty("get")
        #se reduce el tamaño de la imagen
        wpercent = (im_width_half / float(s.size[0]))
        reduced_s = s.resize((im_width_half, im_height_half), Image.ANTIALIAS)
        s_array = np.reshape(np.array(reduced_s), (-1, im_size_half)) #HALF SIZE
        d = 0 # while 0 -> The game continues
        rAll = 0
        j = 0 #Ep_length
        state = (np.zeros([1,h_size]),np.zeros([1,h_size])) #Reset the recurrent layer's hidden state
        #The Q-Network
        epEndTime = addMinute()
        while j < max_epLength: 
            time_next_step = addMicrosecs(1000000/actions_per_second)
            j+=1
            (HP_1, HP_2) = getHealths(s, coordinate_p1, coordinate_p2, length)
            d = is_game_over(HP_1, HP_2, epEndTime)
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < e or total_steps < pre_train_steps:   
                state1 = sess.run(mainQN.rnn_state,\
                        feed_dict={mainQN.scalarInput:[s_array],\
                        mainQN.trainLength:1,mainQN.state_in:state,mainQN.batch_size:1})                
                a = np.random.randint(0,len(actions)) #from our defined actions
            else:
                a, state1 = sess.run([mainQN.predict,mainQN.rnn_state],\
                    feed_dict={mainQN.scalarInput:[s_array],\
                    mainQN.trainLength:1,mainQN.state_in:state,mainQN.batch_size:1})
                a = a[0]
            connection_to_Monty("2"+actions[a-1])
            actionBuffer.append(actions[a-1])
            r = calculate_reward_p2(d, actionBuffer, j, HP_1, HP_2)
            s1 = connection_to_Monty("get")
            s1_reduced = reduced_s = s1.resize((im_width_half, im_height_half), Image.ANTIALIAS)
            s1_array = np.reshape(np.array(s1_reduced), (-1, im_size_half)) #HALF SIZE
            total_steps += 1
            if (d == 0):
                episodeBuffer.append(np.reshape(np.array([s_array,a,r,s1_array,False]),[1,5]))
            else:
                episodeBuffer.append(np.reshape(np.array([s_array,a,r,s1_array,True]),[1,5]))
            if total_steps > pre_train_steps:
                if e > endE:
                    e -= stepDrop

                if total_steps % (update_freq) == 0:
                    updateTarget(targetOps,sess)
                    #Reset the recurrent layer's hidden state
                    state_train = (np.zeros([batch_size,h_size]),np.zeros([batch_size,h_size])) 
                    
                    trainBatch = myBuffer.sample(batch_size,trace_length) #Get a random batch of experiences.
                    #Below we perform the Double-DQN update to the target Q-values
                    Q1 = sess.run(mainQN.predict,feed_dict={\
                        mainQN.scalarInput:[np.vstack(trainBatch[:,3])],\
                        mainQN.trainLength:trace_length,mainQN.state_in:state_train,mainQN.batch_size:batch_size})
                    Q2 = sess.run(targetQN.Qout,feed_dict={\
                        targetQN.scalarInput:[np.vstack(trainBatch[:,3])],\
                        targetQN.trainLength:trace_length,targetQN.state_in:state_train,targetQN.batch_size:batch_size})
                    end_multiplier = -(trainBatch[:,4] - 1)
                    doubleQ = Q2[range(batch_size*trace_length),Q1]
                    targetQ = trainBatch[:,2] + (y*doubleQ * end_multiplier)
                    #Update the network with our target values.
                    sess.run(mainQN.updateModel, \
                        feed_dict={mainQN.scalarInput:[np.vstack(trainBatch[:,0])],mainQN.targetQ:targetQ,\
                        mainQN.actions:trainBatch[:,1],mainQN.trainLength:trace_length,\
                        mainQN.state_in:state_train,mainQN.batch_size:batch_size})
            rAll += r #total reward
            s = s1
            s_array = s1_array
            #sP = s1P
            state = state1
            if d != 0: #probably take it out
                break
            while(datetime.datetime.now()<time_next_step):
                time.sleep(5.0 / 1000.0)
                
        #Add the episode to the experience buffer
        if (j > 8): #MINIMUM NUMBER OF STEPS PER EPISODE
            bufferArray = np.array(episodeBuffer)
            episodeBuffer = list(zip(bufferArray))
            myBuffer.add(episodeBuffer)
            jList.append(j)
            rList.append(rAll)
            if (rAll == 0) and (e < endE):
                e += 0.2
            print("Episode: " + str(i) + " ; Steps: " + str(j) + " ; Total reward: " + str(rAll) + " ; e = " + str(e))
            #Periodically save the model. 
            if i % 50 == 0 and i != 0:
                saver.save(sess,path+'/model-'+str(i)+'.cptk')
                print ("Saved Model")
            if len(rList) % summaryLength == 0 and len(rList) != 0:
                print (total_steps,np.mean(rList[-summaryLength:]), e)
        else:
            i = i-1 #THE EPISODE IS UNDONE 
            print("Insuficient steps in episode.")
    saver.save(sess,path+'/model-'+str(i)+'.cptk')

In [ ]:
print(datetime.datetime.now())

## Empleo de la red entrenada

Se prueba el modelo entrenado

In [1]:
actions_per_second = 2.4
e = 0.01
max_epLength = 60 * actions_per_second #The max allowed length of our episode.
num_episodes = 25 #How many episodes of game environment to train network with.
load_model = True #Whether to load a saved model.
h_size = 512 #The size of the final convolutional layer before splitting it into Advantage and Value streams.

In [ ]:
path = "./drqn_2_02"
#path = "./drqn_2_02_AGGRO"
#path = "./drqn_2_02_KITE"

In [ ]:
(coordinate_p1, coordinate_p2, length) = connection_to_Monty("set")

In [ ]:
tf.reset_default_graph()
#We define the cells for the primary and target q-networks
cell = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
cellT = tf.contrib.rnn.BasicLSTMCell(num_units=h_size,state_is_tuple=True)
mainQN = Qnetwork(h_size,cell,'main', size=0.5)
targetQN = Qnetwork(h_size,cellT,'target', size=0.5)

init = tf.global_variables_initializer()

saver = tf.train.Saver(max_to_keep=5)

#create lists to contain total rewards and steps per episode
jList = []
rList = []
total_steps = 0

path2 = "./Center2"

#Make a path for our model to be saved in.
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
time.sleep(1)
with tf.Session() as sess:
    
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        sess.run(init)

        
    for i in range(num_episodes):
        #Reset environment and get first new observation
        starter_sync() #El proceso terminará a tiempo para que sea un inicio sincronizado
        s = connection_to_Monty("get")
        #se reduce el tamaño de la imagen
        wpercent = (im_width_half / float(s.size[0]))
        reduced_s = s.resize((im_width_half, im_height_half), Image.ANTIALIAS)
        s_array = np.reshape(np.array(reduced_s), (-1, im_size_half)) #HALF SIZE
        d = 0
        #rAll = 0
        j = 0
        margin = 0
        state = (np.zeros([1,h_size]),np.zeros([1,h_size]))
        #The Q-Network
        T1 = datetime.datetime.now()
        while d == 0: #If the agent takes longer than 200 moves to reach either of the blocks, end the trial.
            time_next_step = addMicrosecs(1000000/actions_per_second)
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < e:
                state1 = sess.run(mainQN.rnn_state,\
                    feed_dict={mainQN.scalarInput:[s_array],\
                               mainQN.trainLength:1,mainQN.state_in:state,\
                               mainQN.batch_size:1})
                a = np.random.randint(0,len(actions))
            else:
                a, state1 = sess.run([mainQN.predict,mainQN.rnn_state],\
                    feed_dict={mainQN.scalarInput:[s_array],mainQN.trainLength:1,\
                    mainQN.state_in:state,mainQN.batch_size:1})
                a = a[0]
            connection_to_Monty("2"+actions[a-1]) #MIGHT need some helpt in adjusting this)
            s1 = connection_to_Monty("get")
            (HP_1, HP_2) = getHealths(s1, coordinate_p1, coordinate_p2, length)
            d = is_game_over_neutral_endless(HP_1, HP_2)
            margin = abs(HP_1-HP_2)/200
            s1_reduced = reduced_s = s1.resize((im_width_half, im_height_half), Image.ANTIALIAS)
            s1_array = np.reshape(np.array(s1_reduced), (-1, im_size_half)) #HALF SIZE
            total_steps += 1
            s = s1
            state = state1
            while(datetime.datetime.now()<time_next_step):
                time.sleep(5.0 / 1000.0)
        
        T2 = datetime.datetime.now()
        print("Episode: " + str(i) + " ; LOOSER: " + str(d) + " ; MARGIN: " + str(margin) + " ; Time: " + str(T2-T1))
        #Periodically save the model. 